In [ ]:
!pip install -q onnxruntime
!pip install -q pyannote.audio

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Імпорт необхідних бібліотек
from pyannote.audio import Pipeline
from pyannote.core import Segment, Timeline, Annotation, notebook
from pyannote.database.util import load_rttm
from pyannote.database import get_protocol, FileFinder

from pyannote.metrics.diarization import DiarizationErrorRate, JaccardErrorRate
from pyannote.metrics.identification import IdentificationErrorRate
from pyannote.metrics.detection import DetectionErrorRate

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [ ]:
def read_num_speakers_from_rttm(rttm_file_path):
    speakers = set()
    with open(rttm_file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            # Ідентифікатор спікера зазвичай знаходиться на 8-й позиції (починаючи з 0)
            speaker_id = parts[7]
            speakers.add(speaker_id)
    return len(speakers)

In [ ]:
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.core import Annotation, Segment

def load_rttm(file_path):
    annotation = Annotation()
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            start = float(parts[3])
            duration = float(parts[4])
            label = parts[7]
            annotation[Segment(start, start + duration)] = label
    return annotation

rttm_folder = 'voxconverse_test_rttm'  # Шлях до папки з RTTM файлами
file_names = []  # Список для зберігання назв файлів

# Перебір всіх файлів у папці
for file in os.listdir(rttm_folder):
    if file.endswith('.rttm'):
        # Додавання назви файлу без розширення .rttm до списку
        file_names.append(os.path.splitext(file)[0])

# Створення екземпляра DiarizationErrorRate
der_metric = DiarizationErrorRate(collar=0.5)
jer_metric = JaccardErrorRate(collar=0.5)
ier_metric = IdentificationErrorRate(collar=0.5)

# Масив для зберігання значень DER для кожної пари
error_rates = []
error_rates_jer = []
error_rates_ier = []

for file_name in file_names:
    reference_path = f'voxconverse_test_rttm/{file_name}.rttm'
    hypothesis_path = f'pyannote_output_directory/{file_name}.rttm'

    reference_annotation = load_rttm(reference_path)
    hypothesis_annotation = load_rttm(hypothesis_path)

    # Розрахунок DER для кожної пари
    error_rate = der_metric(reference_annotation, hypothesis_annotation)
    jer = jer_metric(reference_annotation, hypothesis_annotation)
    ier = ier_metric(reference_annotation, hypothesis_annotation)
    error_rates.append(error_rate)
    error_rates_jer.append(jer)
    error_rates_ier.append(ier)

# Розрахунок середнього DER
average_der = sum(error_rates) / len(error_rates)
print(f"Average Diarization Error Rate: {average_der:.2f}")

average_jer = sum(error_rates_jer) / len(error_rates_jer)
print(f"Average jer: {average_jer:.2f}")

average_ier = sum(error_rates_ier) / len(error_rates_ier)
print(f"Average ier: {average_ier:.2f}")


Average Diarization Error Rate: 0.09
Average jer: 0.28
Average ier: 1.01


In [ ]:
df_result = pd.DataFrame([{
    'File': 'Pyannote',
    'DER': average_der,
    'JER': average_jer,
    'IER': average_ier,
}])

display(df_result)

,File,DER,JER,IER
0,Pyannote,0.085988,0.279072,1.014103
